Pandas在Excel表中筛选数据的操作非常简便快捷，在筛选完数据后用`to_excel`会存为单独的Excel文件。如何让筛选好的数据在原Excel文件中新建一个`Sheet`保存呢？那就需要用到`pandas.ExcelWriter`方法。其完整参数如下。

In [ ]:
pandas.ExcelWriter(path, engine=None, date_format=None, datetime_format=None, mode='w', **engine_kwargs)

其中，`path`指xls或者xlsx文件的路径。
<br/>`engine`指用于写入的引擎，若不指定，使用默认引擎。
<br/>`date_format`用于设置日期的格式化字符串，比如‘YYYY-MM-DD’。若有设定，则数据中若有日期的则会按此格式写入。
<br/>`datetime_formatstr`用于设置日期时间的格式化字符串，比如‘YYYY-MM-DD HH:MM:SS’。若有设定，则数据中若有日期时间的则会按此格式写入。
<br/>`mode`用于设置写入模式，有“写入”(`w`)和“添加”(`a`)两种模式，其中默认为“写入”模式。“写入”模式会覆盖原有Excel文件的所有工作表，而“添加”模式则在原有工作表基础上新建表格。如果新建的表格名已存在，则会自动在已存在的表名基础上加上“1”作为新表格的名称。“添加”模式需要版本`0.24.0`及以上才支持，若低于这个版本，则需要升级pandas哦。另外，在使用“添加”模式的时候，需要指定`engine = 'openpyxl'`，不然会报错“ValueError: Append mode is not supported with xlsxwriter!”。

了解以上用法，就可以开始干活了。要求是在原工作簿中新建一个名字为“数量大于500”的新工作表，将“物料表”里的“数量”列中大于500的所有数据单独取出，并存入此工作表。

In [13]:
import pandas as pd
df = pd.read_excel("物料表.xlsx", header  = 2)
df.head()

,请求单号,单位,物料编号,架位,批号,数量,日期
0,A19X9239156,EA,11010XR0073700,LC000001,PC00001,3,2019-01-03
1,A19X9239156,EA,11020XR0025500,LC000002,PC00002,10,2019-01-03
2,A19X9239156,EA,11030XR0013200,LC000003,PC00003,3,2019-01-03
3,A19X9239156,EA,11140XR0000100,LC000004,PC00004,3,2019-01-03
4,A19X9239156,EA,12121XR0172300,LC000005,PC00005,3,2019-01-03


以上先导入`pandas`库，并重命名为`pd`。然后读取Excel工作表，并指定标题行为第3行，即`header=2`。然后用`df.head()`显示前五行数据。完美！

In [14]:
df500 = df[df["数量"]>500]
df500.head()

,请求单号,单位,物料编号,架位,批号,数量,日期
37,A19X9239158,EA,75008XR0000043,LC000038,PC00038,1200,2019-01-03
93,A19X9239160,EA,62010XR050150A,LC000094,PC00094,1000,2019-01-03
102,A19X9239364,EA,62010XR0101044,LC000103,PC00103,720,2019-01-03
110,A19X9239364,EA,62018XR0500101,LC000111,PC00111,540,2019-01-03
123,A19X9239364,EA,63119XR0000160,LC000124,PC00124,540,2019-01-03


以上，筛选出“数量”大于500的所有数据，并存入新的DataFrame“df500”。通过`df500.head()`查看其前五行数据。漂亮！然后就可以调用`ExcelWriter`将数据写入同一个工作簿的新工作表了。此处，我们使用了`a`（添加）模式，然后指定`engine`为`openpyxl`。若保持`engine`为默认，则会报错“ValueError: Append mode is not supported with xlsxwriter!”。另外，数据中有日期数据，所以指定日期时间显示格式为`YYYY-MM-DD`，即只显示年月日，不显示时间。若不指定格式，则日期列的数据会显示成“2019-01-03 00:00:00”。在写入Excel文件的时候，指定表名为“数量大于500”，且不显示行索引，即`index=False`。结果如下。
![](images\result.png)

In [15]:
with pd.ExcelWriter('物料表.xlsx', mode = 'a' ,engine='openpyxl',datetime_format='YYYY-MM-DD') as writer:
    df500.to_excel(writer, sheet_name='数量大于500',index = False)

结果基本满足要求，唯一不好的是，字号不合适，列宽不合适，且数据所在单元格没有边框。没关系，叫老朋友`openpyxl`出来搞一下就好了。

In [17]:
from openpyxl import load_workbook
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
thin = Side(border_style="thin", color="000000")#定义边框粗细及颜色

wb = load_workbook("物料表.xlsx")
ws = wb["数量大于500"]
   
# 调整列宽
ws.column_dimensions['A'].width = 12
ws.column_dimensions['C'].width = 15.5
ws.column_dimensions['G'].width = 10

#设置字号，对齐，缩小字体填充，加边框
for row_number in range(2, ws.max_row+1):
    for col_number in range(1,ws.max_column+1):
        c = ws.cell(row=row_number,column=col_number)
        c.font = Font(size=10)
        c.border = Border(top=thin, left=thin, right=thin, bottom=thin)
        c.alignment = Alignment(horizontal="left", vertical="center")
wb.save("物料表.xlsx")

程序运行完成，迫不及待打开文件看看。
![](images\result1.png)
立马乌鸡变凤凰，有没有？O(∩_∩)O哈哈~